In [3]:
import numpy as np
import pandas as pd
import yaml
import zipfile

In [4]:
# Les données sont trop volumineuses pour être déposées sur Github.
# Solution : chacun de nous définit le chemin vers son répertoire de données dans le fichier
# conf/global_conf NON versionné sur github.
# Utiliser pour le créer le template global_template.yml.
# Le fichier mergé y sera aussi déposé.
file = open('../../conf/global_conf.yml', 'r') 
conf = yaml.safe_load(file)

In [5]:
# Définition d'une fonction pointant vers le répertoire des données
def datapath(fich):
    return conf['data_path'] + fich

print(datapath("users.csv"))

../../data/users.csv


In [6]:
""" 
Changements en 2018 :
Avertissement : Les données sur la qualification de blessé hospitalisé depuis l’année 2018 ne peuvent 
être comparées aux années précédentes suite à des modifications de process de saisie des forces de 
l’ordre. L’indicateur « blessé hospitalisé » n’est plus labellisé par l’autorité de la statistique publique 
depuis 2019.
A partir des données de 2021, les usagers en fuite ont été rajoutés, cela entraîne des manques
d’informations sur ces derniers, notamment le sexe, l’âge, voire la gravité des blessures (indemne, 
blessé léger ou blessé hospitalisé). 

Changements en 2019 :
Les équipements de sécurité jusqu’en 2018 étaient en 2 variables : existence et utilisation.
A partir de 2019, il s’agit de l’utilisation avec jusqu’à 3 équipements possibles pour un même usager 
(notamment pour les motocyclistes dont le port du casque et des gants est obligatoire).
"""

# L'exploration des données montre que les catégories suivantes sont des données manquantes
list_na_values = ['#ERREUR', '(1)']

#------ Lecture des données des caractéristiques à partir de 2019 ------------
#df18 = pd.read_csv(datapath("caracteristiques-2018.csv") )
df19 = pd.read_csv(datapath("caracteristiques-2019.csv"), sep = ";", na_values = list_na_values)
df20 = pd.read_csv(datapath("caracteristiques-2020.csv"), sep = ";", na_values = list_na_values)
df21 = pd.read_csv(datapath("caracteristiques-2021.csv"), sep = ";", na_values = list_na_values)
df22 = pd.read_csv(datapath("caracteristiques-2022.csv"), sep = ";", na_values = list_na_values)

# Num_Acc s'appelle Accident_Id en 2022 : renommer avant de concaténer
df22.rename(columns = {'Accident_Id': 'Num_Acc'}, inplace=True)

# Concaténation des 4 années 
carac =  pd.concat([df19, df20, df21, df22])

# Affichage des caractéristiques du jeu de données
#display(carac.head(10))
#print(carac.info())
#display(carac.describe())

In [7]:
#------ Lecture des données des lieux à partir de 2019 ------------
df19 = pd.read_csv(datapath("lieux-2019.csv"), sep = ";", na_values = list_na_values)
df20 = pd.read_csv(datapath("lieux-2020.csv"), sep = ";", na_values = list_na_values)
df21 = pd.read_csv(datapath("lieux-2021.csv"), sep = ";", na_values = list_na_values)
df22 = pd.read_csv(datapath("lieux-2022.csv"), sep = ";", na_values = list_na_values)

# Concaténation des 4 années et affichage des caractéristiques
lieux =  pd.concat([df19, df20, df21, df22])

#display(lieux.head(10))
#print(lieux.info())
#display(lieux.describe())

In [8]:
#------ Lecture des données des usagers à partir de 2019 ------------
df19 = pd.read_csv(datapath("usagers-2019.csv"), sep = ";", na_values = list_na_values)
df20 = pd.read_csv(datapath("usagers-2020.csv"), sep = ";", na_values = list_na_values)
df21 = pd.read_csv(datapath("usagers-2021.csv"), sep = ";", na_values = list_na_values)
df22 = pd.read_csv(datapath("usagers-2022.csv"), sep = ";", na_values = list_na_values)

# Concaténation des 4 années et affichage des caractéristiques
users =  pd.concat([df19, df20, df21, df22])

#display(users.head(10))
#print(users.info())
#display(users.describe())

In [9]:
#------ Lecture des données des véhicules à partir de 2019 ------------
df19 = pd.read_csv(datapath("vehicules-2019.csv"), sep = ";", na_values = list_na_values)
df20 = pd.read_csv(datapath("vehicules-2020.csv"), sep = ";", na_values = list_na_values)
df21 = pd.read_csv(datapath("vehicules-2021.csv"), sep = ";", na_values = list_na_values)
df22 = pd.read_csv(datapath("vehicules-2022.csv"), sep = ";", na_values = list_na_values)

# Concaténation des 4 années et affichage des caractéristiques
veh =  pd.concat([df19, df20, df21, df22])

#display(veh.head(10))
#print(veh.info())
#display(veh.describe())

In [10]:
#--------- Merge des 4 jeux de données (une ligne par utilisateur) --------------
how = 'inner'
df = pd.merge(users, veh, on = ['Num_Acc', 'id_vehicule', 'num_veh'], how=how) 
df = pd.merge(lieux, df, how=how, on = "Num_Acc")
df = pd.merge(carac, df, how=how, on = "Num_Acc")
df.shape


(494182, 55)

In [11]:
#--------------- Remplacement des -1 par des NA -------------------
# pour la variable actp la valeur 0 est aussi remplacée par NaN (0 – Non renseigné ou sans objet)
df1 = df.replace([-1, "-1", " -1"], np.NaN)
df1["actp"] = df1["actp"].replace("0", np.NaN)
#print(df1["actp"].value_counts(dropna=False))
#print(df1["actp"].dtype)

In [12]:
#df.info()
#------------- sauvegarde du jeu de données complet au format csv dans un répertoire local -----------
fich = conf['local_data_path'] + "accidents.csv"
#print(fich)
df1.to_csv(fich, sep=';', index=False)

In [13]:
fich_zip = conf['local_data_path'] + "accidents.zip"
myZip = zipfile.ZipFile(fich_zip, 'w', zipfile.ZIP_DEFLATED)
myZip.write(fich)
myZip.close()